## Lab5_higaredajra
Benchmarking performance and space efficiency of Pandas DataFrames vs Python Lists with huge data sets.

In [67]:
%%time
# In this cell: Import libraries and declare your cleaning functions.
# These functions can be directly copied from Lab 4 if they were done appropriately.
# These functions must take in a token and return a cleaned version of the token.

# In this cell: import spellcheck library and define your cleaning functions
from spellchecker import SpellChecker

def clean_web_spaces(terms):
    return list(map(lambda s: s.replace("%20", " "), terms))

def remove_second_column(terms):
    for i in range (len(terms)):
        terms[i] = terms[i][0:terms[i].index(",")]
    return terms

def split_spaces(terms):
    for i in range (len(terms)):
        if " " in terms[i]:
            terms.append(terms[i][terms[i].index(" ") + 1:])
            terms[i] = terms[i][:terms[i].index(" ")]
    return terms

def remove_special_chars(token):
    return "".join(char for char in token if char.isalnum())

def remove_all_numeric_terms(token):
    if token.isdigit():
        return ""
    else:
        return token

def make_terms_lowercase(terms):
    for i in range (len(terms)):
        terms[i] = terms[i].lower()
    return terms

spellcheck_dictionary = {}
chk = SpellChecker(distance = 1)

def spellcheck(token):
    if token == "":
        return ""
    elif token in spellcheck_dictionary:
        return spellcheck_dictionary[token]
    else:
        corrected = chk.correction(token)
        spellcheck_dictionary[token] = corrected
        return corrected


CPU times: total: 93.8 ms
Wall time: 97.5 ms


In [68]:
# In this cell: import the dataset, split into individual tokens, create DataFrame, 
# and display the first 20 lines of the DataFrame.
import pandas as pd

terms = list(open("searchTerms.csv", encoding="utf-8"))
column_headings = terms.pop(0)
terms = clean_web_spaces(terms)
terms = remove_second_column(terms)
terms = split_spaces(terms)
terms = make_terms_lowercase(terms)

df = pd.DataFrame()
unclean_terms = pd.Series(terms)
df["uncleaned"] = unclean_terms
df.head(20)


,uncleaned
0,36969
1,cmed
2,kend
3,cmed
4,dync1815h
5,dynd70642
6,dees
7,link
8,7081714
9,kend


In [69]:
# In this cell: Add a new columns to the DataFrame by applying your cleaning 
# functions one at a time. You may break this step into multiple cells if it 
# makes your benchmarking easier. Display the first 20 lines of the DataFrame 
# once it has the four required columns.

df["no_punctuation"] = df.apply(lambda row: remove_special_chars(row["uncleaned"]), axis=1)
df["no_all_nums"] = df.apply(lambda row: remove_all_numeric_terms(row["no_punctuation"]), axis=1)
df["spellchecked"] = df.apply(lambda row: spellcheck(row["no_all_nums"]), axis=1)
df.head(20)

,uncleaned,no_punctuation,no_all_nums,spellchecked
0,36969,36969,,
1,cmed,cmed,cmed,coed
2,kend,kend,kend,kind
3,cmed,cmed,cmed,coed
4,dync1815h,dync1815h,dync1815h,None
5,dynd70642,dynd70642,dynd70642,None
6,dees,dees,dees,does
7,link,link,link,link
8,7081714,7081714,,
9,kend,kend,kend,kind


In [70]:
%%time

# In this cell: export most clean column to list, count terms with Lab 4 dictionary
# counting code, display benchmarking results.
def create_term_freq_dict(terms):
    term_freq = {}
    for term in terms:
        if term not in term_freq:
            term_freq[term] = 1
        else:
            term_freq[term] += 1

    return dict(sorted(term_freq.items(), key=lambda item: item[1], reverse=True))

clean_terms = list(df["spellchecked"])

clean_freq_dict = create_term_freq_dict(clean_terms)
del clean_freq_dict[None]
del clean_freq_dict[""]
print(list(clean_freq_dict.items())[:10])

[('chicken', 18517), ('cheese', 13455), ('beef', 13024), ('cream', 12474), ('sauce', 11034), ('pork', 10987), ('juice', 10924), ('potato', 9775), ('diced', 9524), ('green', 9131)]
CPU times: total: 203 ms
Wall time: 212 ms


**NOTE:** I am putting this md here because the *.value_counts()* table is ridiculously long.
The Wall time for the List approach was 175 ms, while the .value_counts() approach only took about 37 ms, which was almost 5x faster. I'm not completely sure how the Pandas series is structured internally, but it makes sense that the List approach would be relatively slow. The list needs to iterate through all 15 million (or so) items and also keep track of about 24,000 unique keys and count values simultaneously, which is likely also slowing down the process.

In [71]:
%%time

# In this cell: use .value_counts() to do frequency counting. Display benchmarking
# results.
df["spellchecked"].value_counts()


CPU times: total: 46.9 ms
Wall time: 50.4 ms


spellchecked
              163446
chicken        18517
cheese         13455
beef           13024
cream          12474
               ...  
potted             1
wholewheat         1
caked              1
mad                1
alba               1
Name: count, Length: 6661, dtype: int64

**TODO:** Analyze above benchmarking results.

In [72]:
# In this cell: Evaluate space efficiency of python list vs pandas DataFrame.
# Display results. Again, you may break this step into multiple cells if it
# makes your analysis easier.
import sys

print(df["spellchecked"].memory_usage(deep=True))
clean_size = 0
for term in clean_terms:
    clean_size += sys.getsizeof(term)
clean_size += sys.getsizeof(clean_terms)
print(clean_size)

73751154
73751086


I was surprised at the space efficiency of both the pandas DataFrame and the python List. Both require approximately 73 MB of space, but the list is about 70 bytes smaller. I was expecting the DataFrame to be much larger than the list because it seems like there was more optimization internally, which would have required more space.

I think that pandas is more useful in most situations. Even though it tends to use just barely more space in memory, it is often much faster than using normal python lists and can require fewer lines of code to execute the same operations. Those fewer lines of code might be harder to read than a function modifying a list, but they are not obscenely difficult to comprehend after a brief inspection.